# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил(а): Бир Анастасия

### Общая информация
Дата выдачи: 09.04.2020

Дедлайн: 24.04.2020 23:59 MSK

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [106]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [0]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [107]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [108]:
data.shape

(30000, 4)

In [109]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [112]:
from nltk.tokenize import WordPunctTokenizer


tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [113]:
for row in tqdm(X_train):
    for i, item in enumerate(row):
        row[i] = preprocess(row[i])

100%|██████████| 21000/21000 [00:00<00:00, 31008.80it/s]


In [114]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [115]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

In [116]:
for row in X_test:
    for i, item in enumerate(row):
        row[i] = preprocess(row[i])

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов
 - Для каждого примера посчитать, сколько раз каждое слово из словаря в нём встречается


В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите 10000 самых частых слов из `title` и `description` обучающей выборки, отсортируйте их по убыванию частотности.

In [117]:
def to_vocabulary(X_train):
    data = np.reshape(X_train, newshape=(42000,))
    word_list = [word for line in data for word in line.split()]
    bow_vocabulary = {}
    for row in tqdm(word_list):
        if row in bow_vocabulary.keys():
            bow_vocabulary[row] += 1
        else:
            bow_vocabulary[row] = 1
    bow_vocabulary = {k: v for k, v in sorted(bow_vocabulary.items(), key=lambda item: item[1], reverse=True)}
    del data, word_list
    return bow_vocabulary
bow_vocabulary = dict(list(to_vocabulary(X_train).items())[0:10000])  

100%|██████████| 1393309/1393309 [00:00<00:00, 1855138.13it/s]


In [118]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [119]:
def text_to_bow(text: str, bow_vocabulary) -> np.array:
    result = np.array([0]*len(bow_vocabulary))
    for row in text.split():
        if row in list(bow_vocabulary.keys()):
            result[list(bow_vocabulary.keys()).index(row)] += 1
    return result

In [120]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml", bow_vocabulary) != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [121]:
def items_to_bow(items: np.array, bow_vocabulary, use_title=False) -> np.array:
    if use_title == False:
        result = np.zeros((len(items), len(bow_vocabulary)))
        for i in tqdm(range(len(result))):
            result[i] = text_to_bow(items[i][1], bow_vocabulary)
    else:
        result = np.zeros((len(items), len(bow_vocabulary)))
        for i in tqdm(range(len(result))):
            result[i] = text_to_bow(items[i][0] + ' ' + items[i][1], bow_vocabulary)
    return result

In [122]:
assert np.allclose(np.where(items_to_bow([X_train[42]],bow_vocabulary)[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

100%|██████████| 1/1 [00:00<00:00, 63.92it/s]


In [123]:
X_train_bow = items_to_bow(X_train, bow_vocabulary)
X_test_bow = items_to_bow(X_test, bow_vocabulary)

100%|██████████| 9000/9000 [02:53<00:00, 51.95it/s]


### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.

**Подсказка: рассмотрите использование sparse-матриц - в них хранятся только ненулевые элементы.** `scipy.sparse.csr_matrix(X_train_bow)`

In [124]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_bow, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_bow), y_test)
print(acc_lr_bow)

assert acc_lr_bow > 0.7

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7046666666666667


In [125]:
from sklearn.svm import LinearSVC

bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_bow, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_bow), y_test)
print(acc_svc_bow)

assert acc_svc_bow > 0.68

0.6835555555555556


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Модификация признаков (0.5 балла)

Постройте признаковое пространство на конкатенации (через пробел) `title` и `description`.

In [24]:
X_train_mod = items_to_bow(X_train, bow_vocabulary, use_title=True)

100%|██████████| 21000/21000 [07:07<00:00, 49.15it/s] 


In [23]:
X_test_mod = items_to_bow(X_test, bow_vocabulary, use_title=True)

100%|██████████| 9000/9000 [06:00<00:00, 56.88it/s]


### mystem (1.5) балла

Как можно заметить, в текстах одни и те же слова могут быть в разных падежах, а соответственно в bow это будут разные признаки. 
Чтобы исправить это, можно лемматизировать слова - с помощью библиотеки Mystem.

In [154]:
from pymystem3 import Mystem

In [155]:
m = Mystem()

In [27]:
','.join(m.lemmatize("Здравствуйте, ваше благородие"))

'здравствовать,, ,ваш, ,благородие,\n'

__Задание:__ 
* Лемматезируйте `title` и `description` с помощью mystem
* Gостройте BoW-представление
* Обучите на нём логистическую регрессию и SVC.

In [151]:
def mystem_preprocess(text):
    return ' '.join(m.lemmatize(text))

In [29]:
for row in tqdm(X_train):
    for i, item in enumerate(row):
        row[i] = mystem_preprocess(row[i])

100%|██████████| 21000/21000 [00:53<00:00, 391.23it/s]


In [30]:
for row in tqdm(X_test):
    for i, item in enumerate(row):
        row[i] = mystem_preprocess(row[i])

100%|██████████| 9000/9000 [00:25<00:00, 351.93it/s]


In [31]:
bow_mys_vocabulary = dict(list(to_vocabulary(X_train).items())[0:10000])  

100%|██████████| 1431551/1431551 [00:01<00:00, 1122681.10it/s]


In [32]:
X_train_bow_mys = items_to_bow(X_train, bow_mys_vocabulary)
X_test_bow_mys = items_to_bow(X_test, bow_mys_vocabulary)

100%|██████████| 9000/9000 [03:01<00:00, 49.57it/s]


In [33]:
%%time
bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_bow_mys, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_bow_mys), y_test)
print(acc_lr_bow)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7273333333333334
CPU times: user 1min 55s, sys: 2.05 s, total: 1min 57s
Wall time: 1min 4s


In [34]:
%%time
bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_bow_mys, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_bow_mys), y_test)
print(acc_svc_bow)

0.7066666666666667
CPU times: user 5.12 s, sys: 63.2 ms, total: 5.18 s
Wall time: 5.24 s


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

Давайте для простоты считать один tf-idf для title и description. Для каждого слова из bow_vocabulary нужно посчитать:
1. Сколько раз оно встретилось в title и description во всём X_train
2. В тексте скольких товаров встретилось это слово

In [15]:
def concat(items: np.array) -> np.array:
    result = []
    for i in range(len(items)):
        result.append(items[i][0] + ' ' + items[i][1])
    return result

In [16]:
X_train_for_tfidf = concat(X_train)

In [17]:
def create_count_arr(items):
    count_arr = {}
    for key in tqdm(bow_vocabulary.keys()):
        for row in items:
            if key in row.split():
                if key in count_arr.keys():
                    count_arr[key] += 1
                else:
                    count_arr[key] = 1
    return count_arr

In [55]:
count_arr = create_count_arr(X_train_for_tfidf)

100%|██████████| 10000/10000 [18:33<00:00,  8.98it/s]


In [77]:
from math import log

def tf(counter, docs_words_amount):
    return counter / docs_words_amount


def idf(documents_amount, count_arr, word):
    return log(documents_amount / count_arr[word])


def text_to_tfidf(text: str, count_arr: dict, documents_amount: int, bow_vocabulary) -> np.array:
    result = np.array([0]*len(bow_vocabulary))
    for row in text.split():
        num_of_docs = 0
        if row in list(bow_vocabulary.keys()):
            for item in text.split():
                if item == row:
                    num_of_docs += 1
            result[list(bow_vocabulary.keys()).index(row)] = tf(num_of_docs, len(text.split())) * idf(documents_amount, count_arr, row)
    return result

In [80]:
def items_to_tfidf(items: np.array, bow_vocabulary):
    result = np.zeros((len(items), len(bow_vocabulary)))
    for i in range(len(result)):
        result[i] = text_to_tfidf(items[i][1], count_arr, len(items), bow_vocabulary)
    return result

In [83]:
X_train_tfidf = items_to_tfidf(X_train, bow_vocabulary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [126]:
X_test_tfidf = items_to_tfidf(X_test, bow_vocabulary)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [133]:
tfidf_model_lr = LogisticRegression(max_iter=100)
tfidf_model_lr.fit(X_train_tfidf, y_train)
acc_lr_tfidf = accuracy_score(tfidf_model_lr.predict(X_test_tfidf), y_test)
print(acc_lr_tfidf)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7933333333


In [134]:
tfidf_model_svc = LinearSVC(max_iter=70)
tfidf_model_svc.fit(X_train_tfidf, y_train)
acc_svc_tfidf = accuracy_score(tfidf_model_svc.predict(X_test_tfidf), y_test)
print(acc_svc_tfidf)

0.800333339


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [135]:
from sklearn.feature_extraction.text import HashingVectorizer

In [136]:
hash_vector = HashingVectorizer()

In [137]:
X_train_concat = concat(X_train)

In [138]:
X_test_concat = concat(X_test)

In [139]:
X_train_hash = hash_vector.fit_transform(X_train_concat)
X_test_hash = hash_vector.fit_transform(X_test_concat)

In [140]:
%%time
bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_hash, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_hash), y_test)
print(acc_lr_bow)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7236666666666667
CPU times: user 46.5 s, sys: 1.6 s, total: 48.1 s
Wall time: 27.2 s


In [66]:
%%time
bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_hash, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_hash), y_test)
print(acc_svc_bow)

0.83
CPU times: user 3.73 s, sys: 281 ms, total: 4.01 s
Wall time: 4.29 s


#### Word Vectors (1.5 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [68]:
import gensim
from gensim.models.wrappers import FastText

model = FastText.load_fasttext_format('ru.bin')

In [69]:
def sentence_embedding(sentence: str) -> np.array:
    embedding_dim = model['привет'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    for word in sentence.split():
        if word in model:
            features += model[word]
    return features

In [70]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

__Задание:__ 
* Обучите логистическую регрессию и SVM на новых признаках
* Оцените качество (accuracy_score)

In [71]:
def items_embedding(items: np.array, use_title=False) -> np.array:
    result = []
    if use_title == False:
        for i in tqdm(range(len(items))):
            result.append(sentence_embedding(items[i][1]))
    return result

In [72]:
X_train_embedd = items_embedding(X_train)

100%|██████████| 21000/21000 [00:21<00:00, 993.21it/s] 


In [73]:
X_test_embedd = items_embedding(X_test)

100%|██████████| 9000/9000 [00:07<00:00, 1239.80it/s]


In [74]:
%%time
bow_model_lr = LogisticRegression(max_iter=100)
bow_model_lr.fit(X_train_embedd, y_train)
acc_lr_bow = accuracy_score(bow_model_lr.predict(X_test_embedd), y_test)
print(acc_lr_bow)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5603333333333333
CPU times: user 5min 12s, sys: 3.04 s, total: 5min 15s
Wall time: 5min 29s


In [75]:
%%time
bow_model_svc = LinearSVC(max_iter=70)
bow_model_svc.fit(X_train_embedd, y_train)
acc_svc_bow = accuracy_score(bow_model_svc.predict(X_test_embedd), y_test)
print(acc_svc_bow)

0.5143333333333333
CPU times: user 39.6 s, sys: 242 ms, total: 39.8 s
Wall time: 40.6 s


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Что дальше?

Решение каждого пункта 2 балла:

1. N-Gram модели текстовой классификации
    * Признаки - mystem
    * n-gramm'ы - несколько слов подряд объединяются в один токен
    * Модели - только логистическая регрессия
    * Настоятельно рекомендуется использовать sparse-матрицы

In [154]:
from nltk import ngrams

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [156]:
for row in tqdm(X_train):
    for i, item in enumerate(row):
        row[i] = mystem_preprocess(row[i])

100%|██████████| 21000/21000 [00:50<00:00, 416.33it/s]


In [157]:
for row in tqdm(X_test):
    for i, item in enumerate(row):
        row[i] = mystem_preprocess(row[i])

100%|██████████| 9000/9000 [00:23<00:00, 384.56it/s]


In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [164]:
word_vec = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)

In [165]:
X_train_vector = word_vec.fit_transform(concat(X_train))

In [166]:
X_test_vector = word_vec.fit_transform(concat(X_test))

In [167]:
%%time
vec_model_lr = LogisticRegression(max_iter=100)
vec_model_lr.fit(X_train_vector, y_train)
acc_lr_vec = accuracy_score(vec_model_lr.predict(X_test_vector), y_test)
print(acc_lr_vec)

0.20855555555555555
CPU times: user 15.3 s, sys: 105 ms, total: 15.4 s
Wall time: 7.97 s


2. Использовать Vowpal Wabbit вместо sklearn
    * Признаки - обычный BoW title+description 
    * Главный вызов - заставить работать библиотеку и подготовить признаки

3. Другие способы лемматизации (pymorphy2, spaCy)


* Снабжайте код пояснениями;
* Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.

Импортируем библиотеку

In [77]:
import pymorphy2

В pymorphy2 для морфологического анализа слов есть класс MorphAnalyzer. По дефолту уже стоит русский язык

In [78]:
morph = pymorphy2.MorphAnalyzer()

Пишем функцию для лемматизации, будем все приводить к нормальной форме

In [95]:
def pymorph_lem(items):
    result = []
    for row in tqdm(items):
        for item in row:
            line = []
            for word in item.split():
                line.append(morph.parse(word)[0].normal_form)
            result.append(line)
    return result

Ну, собственно применяем функцию

In [96]:
X_train_morph = pymorph_lem(X_train)

100%|██████████| 21000/21000 [03:02<00:00, 114.85it/s]


In [97]:
X_test_morph = pymorph_lem(X_test)

100%|██████████| 9000/9000 [01:16<00:00, 117.68it/s]


Импортируем библиотеку

In [99]:
import spacy

Импортируем русский язык

In [102]:
from spacy.lang.ru import Russian

Создаем объект класса

In [103]:
nlp = Russian()

Пишем функцию для лемматизации

In [141]:
def spacy_lemm(items):
    result = []
    for row in tqdm(items):
        for item in row:
            doc = nlp(item)
            line = [token.lemma_ for token in doc]
            result.append(line)
    return result

Запустили

In [127]:
X_train_spacy = spacy_lemm(X_train)

100%|██████████| 21000/21000 [04:08<00:00, 84.46it/s] 


In [128]:
X_test_spacy = spacy_lemm(X_test)

100%|██████████| 9000/9000 [01:47<00:00, 83.70it/s] 
